# 1) Introduction

Initially, the end-goal of this notebook was to preprocess data for topic detection and tag classification.  
I tried to explain why I choose to diverge or not from a "classical" preprocess on this particular case (see optional & specific).

## 1.1. Étapes de prétraitement

**1) Suppression du bruit**

1. Suppression du formatage  HTML
2. Suppression des contractions
3. La correction orthographique
4. Mettre en minuscule le texte

**2) Suppression des caractères simples**

1. Suppression de la ponctuation, des caractères spéciaux et des nombres
2. Suppression d'un seul caractère (facultatif et spécifique)

**3) Suppression de StopWords**

1. Suppression du mot le plus fréquent
2. Suppression d'un certain type de mot (facultatif et spécifique)

**4) Steming / Lemmatisation**

1. Steming
2. Lemmatisation


Ce pré-processus est utilisé pour effectuer une simple détection de sujet (LDA, NMF, etc.) ou une classification, des informations nécessaires à certaines analyses peuvent être perdues.

 

1.1.1 Vocabulaire
Si vous êtes nouveau dans la PNL, voici une petite liste de concepts utilisés dans ce cahier.

Tokenize: "Processus de conversion d'une chaîne en une liste de sous-chaînes, appelées tokens."

Normalisation du texte: "Processus de transformation d'un texte en une seule forme canonique qu'il n'aurait peut-être pas eu auparavant (par exemple, mettre en miniscule le texte, supprimer les contractions, correction orthographique, stemming / lemmatisation, etc.). La normalisation du texte nécessite de savoir quel type de texte doit être normalisée et comment elle doit être traitée par la suite; il n’existe pas de procédure de normalisation universelle. "

Suppression du bruit: "Processus de suppression de tout élément susceptible d'interférer avec votre analyse (par exemple, suppression du code HTML, mettre en minuscule le texte, suppression de la ponctuation / du caractère spécial, etc.)

Stemming: "Processus de réduction des mots à leur racine , base ou forme de racine - généralement une forme de mot écrit ("fishing", "fished", and "fisher" to the stem "fish")."

Lématisation: "Processus de regroupement des formes fléchies d'un mot afin qu'elles puissent être analysées comme un seul élément, identifié par le lemme du mot, ou par la forme du dictionnaire (ie: "walking" to "walk", "better" to "good")."

StopWord: "Mots qui sont filtrés avant ou après le traitement des données en langage naturel (texte). Les mots d'arrêt font généralement référence aux mots les plus courants dans une langue (des mots comme "The","a", etc. en anglais)."

# 2) Libraries and Dataset 

In [7]:
! pip install bs4
# ! pip install pycontractions # The package has a depencies that have not been updated, so I couldn't use it.
! pip install contractions
! pip install autocorrect 

In [8]:
# generic librairies
import pandas as pd

# Text librairies
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import contractions
from autocorrect import Speller

In [9]:
# https://numpy.org/devdocs/user/basics.types.html

dtypes_questions = {'Id':'int32', 'Score': 'int16', 'Title': 'str', 'Body': 'str'}

In [10]:
%%time
df_questions = pd.read_csv('input/QueryResults first 50000 post.csv',
                           usecols=['Id', 'Score', 'Title', 'Body'], 
                           encoding = "ISO-8859-1",
                           dtype=dtypes_questions,
                           nrows=1000
                          )

CPU times: user 13.8 ms, sys: 4.29 ms, total: 18.1 ms
Wall time: 17.2 ms


In [11]:
df_questions[['Title', 'Body']] = df_questions[[
    'Title', 'Body'
]].applymap(lambda x: str(x).encode("utf-8", errors='surrogatepass').decode(
    "ISO-8859-1", errors='surrogatepass'))

In [12]:
# Remove all questions that have a negative score
df_questions = df_questions[df_questions["Score"] >= 0]

In [49]:
spell = Speller()
token = ToktokTokenizer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
charac = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789'
stop_words = set(stopwords.words("english"))
adjective_tag_list = {'JJ', 'JJR', 'JJS', 'RBR', 'RBS'}  # List of Adjective's tag from nltk package


**Tag list**  
List of tag use in the tagger (pos_tag function) from NLTK:
https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [14]:
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990 entries, 0 to 999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      990 non-null    int32 
 1   Score   990 non-null    int16 
 2   Body    990 non-null    object
 3   Title   990 non-null    object
dtypes: int16(1), int32(1), object(2)
memory usage: 29.0+ KB


# 3) Suppression du bruit

La suppression du bruit consiste à supprimer tout ce qui peut interférer avec votre analyse de texte. C'est comme l'étape de nettoyage des données pour un projet ML classique.

## 3.1. Suppression du code  HTML

In [15]:
df_questions['Body'][10]

'<p>For a table like this:</p>\n\n<pre><code>CREATE TABLE binary_data (\n    id INT(4) NOT NULL AUTO_INCREMENT PRIMARY KEY,\n    description CHAR(50),\n    bin_data LONGBLOB,\n    filename CHAR(50),\n    filesize CHAR(50),\n    filetype CHAR(50)\n);\n</code></pre>\n\n<p>Here is a PHP example:</p>\n\n<pre><code>&lt;?php\n    // store.php3 - by Florian Dittmer &lt;dittmer@gmx.net&gt;\n    // Example php script to demonstrate the storing of binary files into\n    // an sql database. More information can be found at http://www.phpbuilder.com/\n?&gt;\n\n&lt;html&gt;\n    &lt;head&gt;&lt;title&gt;Store binary data into SQL Database&lt;/title&gt;&lt;/head&gt;\n\n    &lt;body&gt;\n        &lt;?php\n            // Code that will be executed if the form has been submitted:\n\n            if ($submit) {\n                // Connect to the database (you may have to adjust\n                // the hostname, username or password).\n\n                mysql_connect("localhost", "root", "password");\n   

In [16]:
%%time

# Parse question and title then return only the text
df_questions['Body'] = df_questions['Body'].apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())
df_questions['Title'] = df_questions['Title'].apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())

CPU times: user 394 ms, sys: 299 µs, total: 394 ms
Wall time: 393 ms


BeautifulSoup nous permet de supprimer efficacement la plupart du code html mais pas tout.

In [17]:
df_questions['Body'][10]

'For a table like this:\nCREATE TABLE binary_data (\n    id INT(4) NOT NULL AUTO_INCREMENT PRIMARY KEY,\n    description CHAR(50),\n    bin_data LONGBLOB,\n    filename CHAR(50),\n    filesize CHAR(50),\n    filetype CHAR(50)\n);\n\nHere is a PHP example:\n<?php\n    // store.php3 - by Florian Dittmer <dittmer@gmx.net>\n    // Example php script to demonstrate the storing of binary files into\n    // an sql database. More information can be found at http://www.phpbuilder.com/\n?>\n\n<html>\n    <head><title>Store binary data into SQL Database</title></head>\n\n    <body>\n        <?php\n            // Code that will be executed if the form has been submitted:\n\n            if ($submit) {\n                // Connect to the database (you may have to adjust\n                // the hostname, username or password).\n\n                mysql_connect("localhost", "root", "password");\n                mysql_select_db("binary_data");\n\n                $data = mysql_real_escape_string(fread(fop

Nous devons donc supprimer le reste ici.

In [18]:
def clean_text(text):
    """ mettre la déf de la fonction"""
    text = re.sub(r"\'", "'", text) # match all literal apostrophe pattern then replace them by a single whitespace
    text = re.sub(r"\n", " ", text) # match all literal Line Feed (New line) pattern then replace them by a single whitespace
    text = re.sub(r"\xa0", " ", text) # match all literal non-breakable space pattern then replace them by a single whitespace
    text = re.sub('\s+', ' ', text) # match all one or more whitespace then replace them by a single whitespace
    text = text.strip(' ')
    return text

In [19]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: clean_text(x)) 
df_questions['Body'] = df_questions['Body'].apply(lambda x: clean_text(x))

CPU times: user 39.9 ms, sys: 456 µs, total: 40.3 ms
Wall time: 41 ms


In [20]:
df_questions['Body'][10]

'For a table like this: CREATE TABLE binary_data ( id INT(4) NOT NULL AUTO_INCREMENT PRIMARY KEY, description CHAR(50), bin_data LONGBLOB, filename CHAR(50), filesize CHAR(50), filetype CHAR(50) ); Here is a PHP example: <?php // store.php3 - by Florian Dittmer <dittmer@gmx.net> // Example php script to demonstrate the storing of binary files into // an sql database. More information can be found at http://www.phpbuilder.com/ ?> <html> <head><title>Store binary data into SQL Database</title></head> <body> <?php // Code that will be executed if the form has been submitted: if ($submit) { // Connect to the database (you may have to adjust // the hostname, username or password). mysql_connect("localhost", "root", "password"); mysql_select_db("binary_data"); $data = mysql_real_escape_string(fread(fopen($form_data, "r"), filesize($form_data))); $result = mysql_query("INSERT INTO binary_data (description, bin_data, filename, filesize, filetype) ". "VALUES (\'$form_description\', \'$data\', \

## 3.2. Suppression des contractions

In [21]:
def expand_contractions(text):
    """expand shortened words, e.g. 'don't' to 'do not'"""
    text = contractions.fix(text)
    return text

In [22]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: expand_contractions(x)) 
df_questions['Body'] = df_questions['Body'].apply(lambda x: expand_contractions(x))

CPU times: user 52.8 ms, sys: 15 µs, total: 52.8 ms
Wall time: 52.6 ms


In [23]:
df_questions['Body'][10]

'For a table like this: CREATE TABLE binary_data ( id INT(4) NOT NULL AUTO_INCREMENT PRIMARY KEY, description CHAR(50), bin_data LONGBLOB, filename CHAR(50), filesize CHAR(50), filetype CHAR(50) ); Here is a PHP example: <?php // store.php3 - by Florian Dittmer <dittmer@gmx.net> // Example php script to demonstrate the storing of binary files into // an sql database. More information can be found at http://www.phpbuilder.com/ ?> <html> <head><title>Store binary data into SQL Database</title></head> <body> <?php // Code that will be executed if the form has been submitted: if ($submit) { // Connect to the database (you may have to adjust // the hostname, username or password). mysql_connect("localhost", "root", "password"); mysql_select_db("binary_data"); $data = mysql_real_escape_string(fread(fopen($form_data, "r"), filesize($form_data))); $result = mysql_query("INSERT INTO binary_data (description, bin_data, filename, filesize, filetype) ". "VALUES (\'$form_description\', \'$data\', \

## 3.3. La correction orthographique

pour 1000 entrées cette correction prends 10 minutes

In [24]:
def autocorrect(text):
    words = token.tokenize(text)
    words_correct = [spell(w) for w in words]
    return ' '.join(map(str, words_correct)) # Return the text untokenize

#df_questions['Title'] = df_questions['Title'].apply(lambda x: autocorrect(x)) 
#df_questions['Body'] = df_questions['Body'].apply(lambda x: autocorrect(x)) 

## 3.4. Mettre en minuscule le texte

Je choisis d'abaisser le texte après le paquet de contractions car celui-ci peut remettre des lettres majuscules lors de la suppression des contractions. La mise en minuscule du texte est une étape classique et utile de la suppression du bruit ou de la normalisation du texte car elle réduit le vocabulaire, normalise le texte et ne coûte presque rien.

In [27]:
%%time

df_questions['Title'] = df_questions['Title'].str.lower()
df_questions['Body'] = df_questions['Body'].str.lower()

CPU times: user 4.13 ms, sys: 11 µs, total: 4.14 ms
Wall time: 3.58 ms


df_questions['Body'][10]

# 4) Suppression des caractères

## 4.1. Suppression de la ponctuation, des caractères spéciaux et des nombres

In [28]:
TOUS les caractères non alphabétiques ont été supprimés (y compris la ponctuation, les nombres et les caractères spéciaux). Ainsi, je ne considère pas les mots importants qui peuvent contenir des caractères spéciaux (comme "C #" en programmation).

SyntaxError: invalid syntax (<ipython-input-28-85914441bee8>, line 1)

In [29]:
def remove_punctuation_and_number(text):
    """remove all punctuation and number"""
    return text.translate(str.maketrans(" ", " ", charac)) 

def remove_non_alphabetical_character(text):
    """remove all non-alphabetical character"""
    text = re.sub("[^a-z]+", " ", text) # remove all non-alphabetical character
    text = re.sub("\s+", " ", text) # remove whitespaces left after the last operation
    return text

In [30]:
df_questions['Body'][11]

'i am looking for the fastest way to obtain the value of ã\x8fâ\x80, as a personal challenge. more specifically, i am using ways that do not involve using #define constants like m_pi, or hard-coding the number in. the program below tests the various ways i know of. the inline assembly version is, in theory, the fastest option, though clearly not portable. i have included it as a baseline to compare against the other versions. in my tests, with built-ins, the 4 * atan(1) version is fastest on gcc 4.2, because it auto-folds the atan(1) into a constant. with -fno-builtin specified, the atan2(0, -1) version is fastest. here is the main testing program (pitimes.c): #include <math.h> #include <stdio.h> #include <time.h> #define iters 10000000 #define testwith(x) { \\ diff = 0.0; \\ time1 = clock(); \\ for (i = 0; i < iters; ++i) \\ diff += (x) - m_pi; \\ time2 = clock(); \\ printf("%s\\t=> %e, time => %f\\n", #x, diff, diffclock(time2, time1)); \\ } static inline double diffclock(clock_t tim

## 4.2. Suppression de la présence d'un seul caractère


Je choisis de supprimer un seul caractère car lorsque nous faisons de la programmation, nous utilisons souvent un seul caractère alphabétique comme nom de variable ("x", "y", "z", etc.). Et j'ai observé que lorsque j'ai essayé de détecter des sujets sans les supprimer, j'ai trouvé beaucoup de sujets avec eux! Et même un sujet que je pourrais nommer "Nom de variable" ...

In [31]:
def remove_single_letter(text):
    """remove single alphabetical character"""
    text = re.sub(r"\b\w{1}\b", "", text) # remove all single letter
    text = re.sub("\s+", " ", text) # remove whitespaces left after the last operation
    text = text.strip(" ")
    return text

In [32]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_single_letter(x)) 
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_single_letter(x)) 

CPU times: user 51.9 ms, sys: 0 ns, total: 51.9 ms
Wall time: 51.6 ms


In [33]:
df_questions['Body'][11]

'am looking for the fastest way to obtain the value of \x8f\x80, as personal challenge. more specifically, am using ways that do not involve using #define constants like m_pi, or hard-coding the number in. the program below tests the various ways know of. the inline assembly version is, in theory, the fastest option, though clearly not portable. have included it as baseline to compare against the other versions. in my tests, with built-ins, the * atan() version is fastest on gcc ., because it auto-folds the atan() into constant. with -fno-builtin specified, the atan2(, -) version is fastest. here is the main testing program (pitimes.): #include <math.> #include <stdio.> #include <time.> #define iters 10000000 #define testwith() { \\ diff = .; \\ time1 = clock(); \\ for ( = ; < iters; ++) \\ diff += () - m_pi; \\ time2 = clock(); \\ printf("%\\=> %, time => %\\", #, diff, diffclock(time2, time1)); \\ } static inline double diffclock(clock_t time1, clock_t time0) { return (double) (time1

# 5) Suppression des stopwords

## 5.1. Removing most frequent words

Supprimer les mots les plus fréquents est une étape classique de la NLP. Les mots les plus fréquents n'ajoutent pas beaucoup d'informations dans la plupart des cas (puisqu'ils sont dans presque toutes les phrases). En les supprimant, vous créez plus d'"espace" pour les autres mots qui peuvent avoir des informations plus utiles.
Vous pouvez utiliser des listes prédéfinies à partir de bibliothèques telles que SciKit-Learn, NLTK et autres. Mais sachez que ces listes peuvent être plus problématiques qu'utiles (en particulier la liste scikit-learn, voir [Stop Word Lists in Free Open-source Software Packages](https://www.aclweb.org/anthology/W18-2502.pdf) pour plus d'informations).

In [34]:
def remove_stopwords(text):
    """remove common words in english by using nltk.corpus's list"""
    words = token.tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    
    return ' '.join(map(str, filtered)) # Return the text untokenize

In [35]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_stopwords(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_stopwords(x)) 

CPU times: user 175 ms, sys: 0 ns, total: 175 ms
Wall time: 174 ms


In [36]:
df_questions['Body'][11]

'looking fastest way obtain value \x8f\x80 , personal challenge. specifically , using ways involve using #define constants like m_pi , hard-coding number in. program tests various ways know of. inline assembly version , theory , fastest option , though clearly portable. included baseline compare versions. tests , built-ins , * atan( ) version fastest gcc . , auto-folds atan( ) constant. -fno-builtin specified , atan2( , - ) version fastest. main testing program ( pitimes. ) : #include <math.> #include <stdio.> #include <time.> #define iters 10000000 #define testwith( ) { \\ diff = . ; \\ time1 = clock( ) ; \\ ( = ; < iters ; ++ ) \\ diff += ( ) - m_pi ; \\ time2 = clock( ) ; \\ printf( " % \\=> % , time => % \\ " , # , diff , diffclock( time2 , time1 ) ) ; \\ } static inline double diffclock( clock_t time1 , clock_t time0 ) { return ( double ) ( time1 - time0 ) / clocks_per_sec ; } int main( ) { int ; clock_t time1 , time2 ; double diff ; / * warmup. atan2 case catches gcc \' atan fold

## 5.2. Removing adjectives (optional)

Je choisis de supprimer les adjectifs en plus de la liste NLTK. Pourquoi? Tout simplement parce que lorsque j'ai d'abord essayé de faire une détection de sujet dans un cahier suivant celui-ci et cela améliore ma détection de sujet. Je pensais aussi que les adjectifs n'ajouteraient aucune information utile. En même temps, je pourrais aussi supprimer des verbes avec le même raisonnement. Mais je ne l'ai pas fait parce que l'ensemble de données StackOverflow concerne la programmation. Et en programmation, nous avons beaucoup de verbes, ou de mots qui peuvent être interprétés comme un verbe, qui peuvent être importants ("return", "get", "request", "replace", etc.). 

In [37]:

def remove_by_tag(text, undesired_tag):
    """remove all words by using ntk tag (adjectives, verbs, etc.)"""
    words = token.tokenize(text) # Tokenize each words
    words_tagged = nltk.pos_tag(tokens=words, tagset=None, lang='eng') # Tag each words and return a list of tuples (e.g. ("have", "VB"))
    filtered = [w[0] for w in words_tagged if w[1] not in undesired_tag] # Select all words that don't have the undesired tags
    
    return ' '.join(map(str, filtered)) # Return the text untokenize

In [38]:
%%time
df_questions['Title'] = df_questions['Title'].apply(
    lambda x: remove_by_tag(x, adjective_tag_list))
df_questions['Body'] = df_questions['Body'].apply(
    lambda x: remove_by_tag(x, adjective_tag_list))

CPU times: user 2.98 s, sys: 38.3 ms, total: 3.02 s
Wall time: 3.02 s


In [39]:
df_questions['Body'][11]

'looking way obtain value \x8f\x80 , challenge. specifically , using ways involve using constants like m_pi , number program tests ways know inline assembly version , theory , option , though clearly portable. included baseline compare versions. tests , built-ins , * atan( ) version gcc . , atan( ) constant. -fno-builtin specified , atan2( , - ) version fastest. testing program ( pitimes. ) : #include <math.> #include <stdio.> #include <time.> #define iters 10000000 #define testwith( ) { diff = . ; \\ time1 = clock( ) ; \\ ( = ; iters ; ++ ) \\ += ( ) - m_pi ; \\ time2 = clock( ) ; \\ printf( " % \\=> % , time => % \\ " , # , diff , diffclock( time2 , time1 ) ) ; \\ } inline diffclock( clock_t time1 , clock_t time0 ) { return ( ) ( - time0 ) / clocks_per_sec ; } int main( ) { int ; clock_t time1 , time2 ; diff ; / * warmup. atan2 case catches gcc \' folding ( would * optimise ` ` * atan( ) - m_pi \' \' no-op ) , * */ testwith( * atan( ) ) testwith( * atan2( , ) ) #if __gnuc__ ) &&amp ;

# 6) Stemming / Lemmatisation

Le Stemming et la Lemmatisation sont des opérations qui:
- peuvent améliorer votre temps de calcul en réduisant votre vocabulaire
- aider à généraliser plus facilement en regroupant les mots (ex: "suis", "sont", "être", etc. seront transformés en "être" pour la lemmatisation)

## 6.1. Stemming

Je n'ai pas choisi d'utiliser le stemming ici, mais l'on doit toujours envisager cette alternative car elle est beaucoup moins coûteuse.

Le stemming est le processus de réduction des mots fléchis à leur racine mot, base ou forme de racine - généralement une forme de mot écrit ("fishing", "fished", and "fisher" à la racine "fish"). Il fonctionne généralement en supprimant l'affixe d'un mot. Un affixe peut être un suffixe ou un préfixe (par exemple «-ed», «-ing», etc.). C'est simple mais ne fonctionnera pas lorsque le mot est "irrégulier" ("ran" et "run"). C'est une opération plus simple que la lemmatisation, qui peut suffire dans certains cas, mais peut faire trop d'erreurs dans d'autres cas.

In [40]:
words = ["program", "programs", "programer", "programing", "programers"]
  
for w in words:
    print(w, " : ", stemmer.stem(w))

program  :  program
programs  :  program
programer  :  program
programing  :  program
programers  :  program


In [41]:
def stem_text(text):
    """Stem the text"""
    words = nltk.word_tokenize(text) # tokenize the text then return a list of tuple (token, nltk_tag)
    stem_text = []
    for word in words:
        stem_text.append(stemmer.stem(word)) # Stem each words
    return " ".join(stem_text) # Return the text untokenize

In [42]:
# %%time

# df_questions['Title'] = df_questions['Title'].apply(lambda x: stem_text(x)) 
# df_questions['Body'] = df_questions['Body'].apply(lambda x: stem_text(x)) 

## 6.2. Lemmatization

Comme dit au début, la lemmatisation est le processus de remplacement d'un mot par son lemma (forme canonique ou forme dictionnaire). Mais dans certains cas, un lemmatiseur peut ne pas être en mesure de trouver la bonne racine si vous ne précisez pas le type de mot comme vous pouvez le voir ci-dessous.

In [43]:
print(lemmatizer.lemmatize("stripes", "v"))
print(lemmatizer.lemmatize("stripes", "n"))  
print(lemmatizer.lemmatize("are"))
print(lemmatizer.lemmatize("are", "v"))

strip
stripe
are
be


Une façon de contourner ce problème consiste à utiliser un marqueur et à passer le type de mot dans la fonction lemmatize. MAIS c'est vraiment coûteux. La mise en tige ou une simple lemmatisation à cet égard est bien plus efficace. 

In [44]:
def lemmatize_text(text):
    """Lemmatize the text by using tag """
    
    tokens_tagged = nltk.pos_tag(nltk.word_tokenize(text))  # tokenize the text then return a list of tuple (token, nltk_tag)
    lemmatized_text = []
    for word, tag in tokens_tagged:
        if tag.startswith('J'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'a')) # Lemmatisze adjectives. Not doing anything since we remove all adjective
        elif tag.startswith('V'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'v')) # Lemmatisze verbs
        elif tag.startswith('N'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'n')) # Lemmatisze nouns
        elif tag.startswith('R'):
            lemmatized_text.append(lemmatizer.lemmatize(word,'r')) # Lemmatisze adverbs
        else:
            lemmatized_text.append(lemmatizer.lemmatize(word)) # If no tags has been found, perform a non specific lemmatization
    return " ".join(lemmatized_text) # Return the text untokenize

In [45]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: lemmatize_text(x)) 
df_questions['Body'] = df_questions['Body'].apply(lambda x: lemmatize_text(x)) 

CPU times: user 2.98 s, sys: 11.3 ms, total: 2.99 s
Wall time: 2.99 s


In [46]:
df_questions['Body'][11]

"look way obtain value \x8f\x80 , challenge . specifically , use way involve use constant like m_pi , number program test way know inline assembly version , theory , option , though clearly portable . include baseline compare version . test , built-ins , * atan ( ) version gcc . , atan ( ) constant . -fno-builtin specify , atan2 ( , - ) version fast . test program ( pitimes . ) : # include < math. > # include < stdio. > # include < time. > # define iters 10000000 # define testwith ( ) { diff = . ; \\ time1 = clock ( ) ; \\ ( = ; iters ; ++ ) \\ += ( ) - m_pi ; \\ time2 = clock ( ) ; \\ printf ( `` % \\= > % , time = > % \\ `` , # , diff , diffclock ( time2 , time1 ) ) ; \\ } inline diffclock ( clock_t time1 , clock_t time0 ) { return ( ) ( - time0 ) / clocks_per_sec ; } int main ( ) { int ; clock_t time1 , time2 ; diff ; / * warmup . atan2 case catch gcc ' folding ( would * optimise ` ` * atan ( ) - m_pi ' ' no-op ) , * * / testwith ( * atan ( ) ) testwith ( * atan2 ( , ) ) # if __gnuc

# 7) Feature engineering

L'utilisation du titre et du corps en même temps donne de bien meilleurs résultats pour la détection des sujets.

In [47]:
df_questions['Text'] = df_questions['Title'] + ' ' + df_questions['Body']

# 8) Exportation des données

In [48]:
df_questions.to_csv('df_questions_fullclean.csv', encoding='utf-8', errors='surrogatepass')